# Kobling og dublettkontroll
Dette er fra grunnkurset med noen nye tillegg.

In [ ]:
import numpy as np
import pandas as pd
from io import StringIO 

## Koble filer
Vi bruker merge når vi skal koble filer. Denne fungerer som en sql join. Det betyr at alle kobler med alle med samme nøkkel. Vi kan velge om vi skal ta med de som ikke kobler.

Vi starter med å lese inn 2 filer som, når de kobles, inneholder alle varianter av koblinger. Det er disse variantene:
- 1:1
- 1:mange
- mange:1
- mange:mange
- 1:ingen
- mange:ingen
- ingen:1
- ingen:mange



In [ ]:
reg1file="""
3 c 03
1 a
2 b 02
4 e 05
3 d 04
4 f 06
4 g 07
7 i 09
5 h 08
7 j 10
"""
reg1 = pd.read_csv(
        StringIO(reg1file),
        names=['id', 'mstat', 'county'],
        dtype=object,
        header=None,
        sep=' '
    )

In [ ]:
reg2file="""
2 y 19
1 z 20
2 x 18
4 v 06
3 w 17
4 u 15
8 s 12
6 t 14
8 r 11
"""
reg2 = pd.read_csv(
        StringIO(reg2file),
        names=['id', 'cstat', 'county'],
        dtype=object,
        header=None,
        sep=' '
    )

In [ ]:
display(reg1, reg2)

## Standard merge (inner join)
Vi starter med en standard merge. Den tilsvarer en inner join i sql. Vi ser at variabler med samme navn i de dataframes som kobles kommer med med hvert sitt nye variabelnavn

In [ ]:
pd.merge(reg1, reg2, on='id')

Resultatet ble ikke sortert på nøkkelen id. Vi kan legge til det med sort_values

In [ ]:
pd.merge(reg1, reg2, on='id').sort_values('id')

Nøklene kan ha forskjellige navn. Da angir vi navnet på nøkkelvariablene

In [ ]:
pd.merge(reg1, 
         reg2, 
         left_on='id', 
         right_on='id'
        ).sort_values('id')

## Full join
Vi kan gjøre en full join. Da vil alle som ikke kobler også komme med i resultatet. Vi angir koblingstypen med how-parameteren. Det er også lurt å ta med en indikator slik at vi kan se hvor de enkelte radene kommer fra. Variabler som har samme navn i begge filene uten å være en del av nøklene får nye navn: suffiks _x og _y

In [ ]:
pd.merge(reg1, 
         reg2, 
         on='id', 
         how='outer', 
         indicator=True
        ).sort_values('id')

Vi kan velge suffiks selv

In [ ]:
pd.merge(reg1, 
         reg2, 
         on='id', 
         how='outer', 
         indicator=True, 
         suffixes = ('_1', '_2')
        ).sort_values('id')

## Left join

In [ ]:
pd.merge(reg1, 
         reg2, 
         on='id',
         how='left', 
         indicator=True
        ).sort_values('id')

## Right join

In [ ]:
pd.merge(reg1, 
         reg2, 
         on='id', 
         how='right', 
         indicator=True
        ).sort_values('id')

## Lagre resultatet
Vi kan lagre koblingen i en panda dataframe

In [ ]:
left = pd.merge(reg1, 
                reg2, 
                on='id', 
                how='left', 
                indicator=True
               ).sort_values('id')\
                .copy()
left

## Sjekke forekomster
Hvis vi bare vil se hvilke identer som er i reg2 uten å hente noe data derfra kan vi bruke et oppslag med isin og bare returnere de som er der (= True)

In [ ]:
reg1[reg1['id'].isin(reg2['id'])]

Vi kan gjøre det motsatte med masken ved å bruke "~" og finne de som er i reg1, men ikke i reg2.

In [ ]:
reg1[~reg1['id'].isin(reg2['id'])]

## Sette sammen dataene under hverandre (append = concat)
Vi kan også sette sammen dataene våre etter hverandre istedenfor å koble dem. Vi kan bruke concat til det. Vi ser at variabler som ikke er i begge datasett får NaN-verdi fra det datasettet de ikke er i

In [ ]:
pd.concat([reg1, reg2])

## Dublettsjekk
Vi starter med å liste alle like rader. Først lager vi et datasett som har like rader, deretter bruker vi duplicated til å fjerne dem

In [ ]:
reg1_4 = reg1[reg1['id'] == '4'].copy()
reg1_4

In [ ]:
reg1dubl = pd.concat([reg1, reg1_4])
reg1dubl

In [ ]:
reg1dubl[~reg1dubl.duplicated()]

Nå skal vi liste dubletter på id-variabelen i det opprinnelige reg1-datasettet. Vi sorterer resultatet slik at det blir lettere å tolke resultatet.
Her blir alle som ikke er første dublett listet

In [ ]:
reg1[reg1.duplicated(['id'], keep='first')].sort_values(['id'])

Her blir alle unntatt den siste listet

In [ ]:
reg1[reg1.duplicated(['id'], keep='last')].sort_values(['id'])

Nå lister vi alle dubletter på id-variabelen

In [ ]:
reg1[reg1.duplicated(['id'], keep=False)].sort_values(['id'])

Vi kan velge å beholde kun den første dubletten for hver id på datasettet

In [ ]:
reg1[~reg1.duplicated(['id'], keep='first')]

Vi kan velge å beholde kun den siste dubletten for hver id på datasettet

In [ ]:
reg1[~reg1.duplicated(['id'], keep='last')]